# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [30]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [31]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	authors
Speaker statistical averageness modulates word recognition in adverse listening conditions.	Talk	icphs-2019	ICPhS 2019	2019-08-08	Melbourne, Australia		Schuerman, W. L., McQueen, J. M., & Meyer, A.
Sparking neural plasticity	Talk	postdocslam-2018	UCSF Postdoc Slam	2018-11-19	San Francisco, California	https://www.youtube.com/watch?v=cQyWMwB2Wig&t=18s
Transcutaneous vagus nerve stimulation enhances non-native speech categorization.	Poster	nbl-2018	Tenth Annual Meeting of the Society for the Neurobiology of Language	2018-08-16	Quebec City, Canada		Fernando Llanos, Jacie R. McHaney, Matthew K. Leonard, William L. Schuerman, Han G. Yi, Bharath Chandrasekaran
Be like me or be like everyone: Effects of phonetic prototypicality on speech recognition.	Talk	mpi-2016	MPI Proudly Presents	2016-06-01	Nijmegen, the Netherlands		W. L. Schuerman
Praatalign: Phonetic alignment made easier for psycholinguistic data processing.	Poster	amlap-2015	AM

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [32]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,authors
0,Speaker statistical averageness modulates word...,Talk,icphs-2019,ICPhS 2019,2019-08-08,"Melbourne, Australia",NaN,"Schuerman, W. L., McQueen, J. M., & Meyer, A."
1,Sparking neural plasticity,Talk,postdocslam-2018,UCSF Postdoc Slam,2018-11-19,"San Francisco, California",https://www.youtube.com/watch?v=cQyWMwB2Wig&t=18s,NaN
2,Transcutaneous vagus nerve stimulation enhance...,Poster,nbl-2018,Tenth Annual Meeting of the Society for the Ne...,2018-08-16,"Quebec City, Canada",NaN,"Fernando Llanos, Jacie R. McHaney, Matthew K. ..."
3,Be like me or be like everyone: Effects of pho...,Talk,mpi-2016,MPI Proudly Presents,2016-06-01,"Nijmegen, the Netherlands",NaN,W. L. Schuerman
4,Praatalign: Phonetic alignment made easier for...,Poster,amlap-2015,AMLaP XXI,2015-08-03,"Valleta, Malta",NaN,"Schuerman, W., Torreira, F., & Lubbers, M."
5,Changes in consonant perception driven by adap...,Poster,icphs-2015,ICPhS 2015,2015-08-10,"Glasgow, Scotland",NaN,"Schuerman, W., Nagarajan, S., & Houde, J."
6,Speaker intelligibility may influence percepti...,Talk,bias-2014,Bias in Auditory Perception,2014-09-19,"Aarhus, Denmark",NaN,"Schuerman, W., Meyer, A., & McQueen, J."
7,Listeners recognize others’ speech better than...,Poster,amlap-2014,AMLaP XX,2014-09-03,"Edinburgh, Scotland",NaN,"Schuerman, W., Meyer, A., & McQueen, J."
8,Do listeners recognize others’ speech better t...,Talk,mpi-2014,MPI Proudly Presents,2014-06-12,"Nijmegen, the Netherlands",NaN,"Schuerman, W., Meyer, A., & McQueen, J."
9,Development of vowel spaces from age 21 to age...,Talk,interdisciplinary-2013,5th International and Interdisciplinary Resear...,2013-10-06,"Indiana University, Bloomington",NaN,"Lutzross, A., Schuerman, W., Sprouse, R., & Ga..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [33]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [34]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.authors)) > 3:
        md += "\n" + html_escape(item.authors) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [35]:
!ls ../_talks

2011-05-14-young-2011.md             2015-08-03-amlap-2015.md
2012-09-06-amplap-2012.md            2015-08-10-icphs-2015.md
2012-09-09-interspeech-2012.md       2016-06-01-mpi-2016.md
2013-10-06-interdisciplinary-2013.md 2018-08-16-nbl-2018.md
2014-06-12-mpi-2014.md               2018-11-19-postdocslam-2018.md
2014-09-03-amlap-2014.md             2019-08-08-icphs-2019.md
2014-09-19-bias-2014.md


In [18]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
